In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import root_mean_squared_error

In [77]:
data= pd.read_csv('C:/Users/visco/Downloads/playground-series-s6e1/train.csv')
tdata= pd.read_csv('C:/Users/visco/Downloads/playground-series-s6e1/test.csv')

data.columns
print(data.shape)

(630000, 13)


In [63]:
#Missing Value
data_hilang = data.isnull().sum()
print(data_hilang)

id                  0
age                 0
gender              0
course              0
study_hours         0
class_attendance    0
internet_access     0
sleep_hours         0
sleep_quality       0
study_method        0
facility_rating     0
exam_difficulty     0
exam_score          0
dtype: int64


In [82]:
y= data['exam_score']
X= data.drop(['id','exam_score','gender','course'],axis=1)

In [65]:
for col in X:
    tipe= X[col].dtype
    print(f'{col} : {tipe}')

age : int64
gender : object
course : object
study_hours : float64
class_attendance : float64
internet_access : object
sleep_hours : float64
sleep_quality : object
study_method : object
facility_rating : object
exam_difficulty : object


In [83]:
#feature engineering
X['internet_access']= X['internet_access'].map({'yes': 1, 'no': 0}) 
X['sleep_quality']= X['sleep_quality'].map({'poor':0, 'average': 5, 'good':10})
X['facility_rating']= X['facility_rating'].map({'low':0, 'medium': 5, 'high':10})
X['exam_difficulty']= X['exam_difficulty'].map({'easy':0, 'moderate': 5, 'hard':10})

# X=pd.get_dummies(X, columns= ['course'])
X= pd.get_dummies(X, columns= ['study_method'])
fitur_diskrit= X.dtypes == int

In [ ]:
#

In [67]:
#mi score
def make_mi_scores(X,y,discreate_feature):
    mi_scores= mutual_info_regression(X,y,discrete_features=discreate_feature) #masukkan data feature importany
    mi_scores= pd.Series(mi_scores, name= 'MI Scores', index= X.columns)
    mi_scores= mi_scores.sort_values(ascending=False) #urutkan dari yang terbesar
    return mi_scores

mi_scores= make_mi_scores(X,y,fitur_diskrit)
mi_scores[::]

study_hours                   0.912392
class_attendance              0.302023
sleep_hours                   0.070280
sleep_quality                 0.037503
facility_rating               0.030249
study_method_coaching         0.023369
age                           0.011865
study_method_self-study       0.010686
exam_difficulty               0.006852
study_method_mixed            0.006799
study_method_group study      0.005732
study_method_online videos    0.005338
gender_other                  0.004523
course_bca                    0.004151
course_b.tech                 0.003915
course_b.sc                   0.003404
course_b.com                  0.003301
internet_access               0.002137
course_bba                    0.001868
gender_female                 0.001739
course_diploma                0.001445
gender_male                   0.001246
course_ba                     0.000893
Name: MI Scores, dtype: float64

In [85]:
#model
train_X,val_X,train_y,val_y= train_test_split(X,y, test_size=0.2,random_state=1)

model = RandomForestRegressor(n_estimators=300,max_depth=17,min_samples_leaf= 30,random_state=1)
model.fit(train_X,train_y)
val_pred= model.predict(val_X)
print('depth: 17, leaf: 30')
print(root_mean_squared_error(val_y,val_pred))

    

    

depth: 17, leaf: 30
8.88431796178178
